In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from persiantools.jdatetime import JalaliDate

In [2]:
BOURS_API = "https://cdn.tsetmc.com/api/MarketData/GetMarketValueByFlow/1/9999"
FARABOURS_API = "https://cdn.tsetmc.com/api/MarketData/GetMarketValueByFlow/2/9999"

In [ ]:
def make_df(api_url):
    json_data = requests.get(api_url).json()
    dates = [item["deven"] for item in json_data["marketValue"]]
    market_values = [item["marketCap"] for item in json_data["marketValue"]]

    df = pd.DataFrame({
    "date": pd.to_datetime(dates, format="%Y%m%d"),
    "market_value": market_values,
    }).sort_values("date").reset_index(drop=True)
    
    df["date"] = df["date"].apply(
        lambda x: JalaliDate.to_jalali(x).strftime("%Y-%m-%d"))
    
    return df

def sma_and_bullish_cross(api_url):
    df = make_df(api_url)
    
    df["mv20"] = df["market_value"].rolling(20).mean()    # 20-day moving average
    df["mv60"] = df["market_value"].rolling(60).mean()    # 60-day moving average
    df["mv240"] = df["market_value"].rolling(240).mean()  # 240-day moving average
    
    df = df.dropna()
    df["bullish_cross"] = (
    (df["mv20"] > df["mv60"]) &
    (df["mv20"].shift(1) <= df["mv60"].shift(1)))
    
    return df